In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# Maps of facilities, tailings and land footprints

In [ ]:
from visualization_functions import plot_polygons_assginment

In [ ]:
plot_polygons_assginment(
    main_gdf,
    tailings_gdf,
    polygon_gdf,  # Make sure polygon_gdf already has 'main_id' and 'tailing_id'
    output_html="facility_tailing_assignment_map.html"
)

# Number of commodities covered

In [9]:
main_gdf = pd.read_csv("data/Tables/main_gdf.csv")
main_gdf

,main_id,facility_name,facility_group_name,facility_group_id,reported_company,company_name_folder,company_id,longitude,latitude,geometry,...,company_URL,company_facility_URL,MDO_URL,recent_transaction,owner(s),operator(s),MAC_member,aggregation_level,allocation,comment
0,QC-MAIN-5801b453,Abcourt-Barvue,NaN,NaN,Abcourt Mines Inc.,Abcourt Mines Inc,CMP-0a6a3062,-77.679010,48.521540,POINT (-77.67901 48.52154),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
1,QC-MAIN-d29e0839,Sleeping Giant,NaN,NaN,Abcourt Mines Inc.,Abcourt Mines Inc,CMP-0a6a3062,-77.974340,49.132590,POINT (-77.97434 49.13259),...,https://abcourt.info/,https://abcourt.info/projects/the-sleeping-gia...,https://miningdataonline.com/property/1589/Sle...,NaN,Abcourt Mines Inc (100%),Abcourt Mines Inc,No,x,NaN,No environmental data.
2,QC-MAIN-f1ff4920,Akasaba West,NaN,NaN,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-77.580000,48.040000,POINT (-77.58 48.04),...,https://agnicoeagle.com/Home/default.aspx,https://www.agnicoeagle.com/English/operations...,https://miningdataonline.com/property/47/Meado...,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3,NU-MAIN-4ea8ac9d,Amaruq,Meadowbank complex,GRP-0a2c0d69,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-96.697000,65.415000,POINT (-96.697 65.415),...,https://agnicoeagle.com/Home/default.aspx,https://www.agnicoeagle.com/English/operations...,https://miningdataonline.com/property/47/Meado...,NaN,Agnico Eagle Mines Limited (100%),Agnico Eagle Mines Limited,Yes,Site-specific,NaN,Very good reporting with data for 2023.
4,QC-MAIN-e7e6a960,Canadian Malartic,NaN,NaN,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-78.130824,48.122223,POINT (-78.130824 48.122223),...,https://agnicoeagle.com/Home/default.aspx,https://malartic.agnicoeagle.com/en/,https://miningdataonline.com/property/20/Canad...,NaN,Agnico Eagle Mines Limited (100%),Canadian Malartic Corporation,Yes,Site-specific,NaN,Very good reporting with data for 2023.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,ON-MAIN-c5fefb01,Mishi,NaN,NaN,Wesdome Gold Mines Ltd.,Wesdome Gold Mines Ltd,CMP-3d2c4955,-85.452500,48.110280,POINT (-85.4525 48.11028),...,https://www.wesdome.com/English/overview/defau...,x,https://miningdataonline.com/property/1238/Mis...,Closed,x,x,No,Site-specific,NaN,Very good reporting for 2023.
266,BC-MAIN-feb13c30,Record Ridge,NaN,NaN,West High Yield Resources Ltd.,West High Yield Resources Ltd,CMP-1023f9c0,-117.884000,49.082000,POINT (-117.884 49.082),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
267,YT-MAIN-5e0d8b48,Casino,NaN,NaN,Western Copper and Gold Corporation,Western Copper and Gold Corporation,CMP-2dac9896,-138.833330,62.733330,POINT (-138.83333 62.73333),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
268,ON-MAIN-12c68d49,Eagle's Nest,NaN,NaN,Wyloo,Wyloo,CMP-65b36728,-86.303800,52.742020,POINT (-86.3038 52.74202),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN


In [8]:
main_gdf

,main_id,facility_name,facility_group_name,facility_group_id,reported_company,company_name_folder,company_id,longitude,latitude,geometry,...,company_URL,company_facility_URL,MDO_URL,recent_transaction,owner(s),operator(s),MAC_member,aggregation_level,allocation,comment
0,QC-MAIN-5801b453,Abcourt-Barvue,NaN,NaN,Abcourt Mines Inc.,Abcourt Mines Inc,CMP-0a6a3062,-77.679010,48.521540,POINT (-77.67901 48.52154),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
1,QC-MAIN-d29e0839,Sleeping Giant,NaN,NaN,Abcourt Mines Inc.,Abcourt Mines Inc,CMP-0a6a3062,-77.974340,49.132590,POINT (-77.97434 49.13259),...,https://abcourt.info/,https://abcourt.info/projects/the-sleeping-gia...,https://miningdataonline.com/property/1589/Sle...,NaN,Abcourt Mines Inc (100%),Abcourt Mines Inc,No,x,NaN,No environmental data.
2,QC-MAIN-f1ff4920,Akasaba West,NaN,NaN,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-77.580000,48.040000,POINT (-77.58 48.04),...,https://agnicoeagle.com/Home/default.aspx,https://www.agnicoeagle.com/English/operations...,https://miningdataonline.com/property/47/Meado...,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3,NU-MAIN-4ea8ac9d,Amaruq,Meadowbank complex,GRP-0a2c0d69,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-96.697000,65.415000,POINT (-96.697 65.415),...,https://agnicoeagle.com/Home/default.aspx,https://www.agnicoeagle.com/English/operations...,https://miningdataonline.com/property/47/Meado...,NaN,Agnico Eagle Mines Limited (100%),Agnico Eagle Mines Limited,Yes,Site-specific,NaN,Very good reporting with data for 2023.
4,QC-MAIN-e7e6a960,Canadian Malartic,NaN,NaN,Agnico Eagle Mines Limited,Agnico Eagle Mines Limited,CMP-6265c407,-78.130824,48.122223,POINT (-78.130824 48.122223),...,https://agnicoeagle.com/Home/default.aspx,https://malartic.agnicoeagle.com/en/,https://miningdataonline.com/property/20/Canad...,NaN,Agnico Eagle Mines Limited (100%),Canadian Malartic Corporation,Yes,Site-specific,NaN,Very good reporting with data for 2023.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,ON-MAIN-c5fefb01,Mishi,NaN,NaN,Wesdome Gold Mines Ltd.,Wesdome Gold Mines Ltd,CMP-3d2c4955,-85.452500,48.110280,POINT (-85.4525 48.11028),...,https://www.wesdome.com/English/overview/defau...,x,https://miningdataonline.com/property/1238/Mis...,Closed,x,x,No,Site-specific,NaN,Very good reporting for 2023.
266,BC-MAIN-feb13c30,Record Ridge,NaN,NaN,West High Yield Resources Ltd.,West High Yield Resources Ltd,CMP-1023f9c0,-117.884000,49.082000,POINT (-117.884 49.082),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
267,YT-MAIN-5e0d8b48,Casino,NaN,NaN,Western Copper and Gold Corporation,Western Copper and Gold Corporation,CMP-2dac9896,-138.833330,62.733330,POINT (-138.83333 62.73333),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
268,ON-MAIN-12c68d49,Eagle's Nest,NaN,NaN,Wyloo,Wyloo,CMP-65b36728,-86.303800,52.742020,POINT (-86.3038 52.74202),...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN


In [10]:
raw_counts = main_gdf[["facility_type", "commodities"]].dropna()
raw_counts_grouped = raw_counts.value_counts().reset_index(name="count")
raw_counts_grouped

,facility_type,commodities,count
0,mining,Gold,26
1,manufacturing,Steel,14
2,mining,"Gold, silver",11
3,mining,Iron,8
4,project,Lithium (spodumene),7
...,...,...,...
135,mining,"Nickel, copper, cobalt",1
136,mining,"Nickel, cobalt, copper, platinum group metals,...",1
137,mining,Lithium,1
138,mining,Ilmenite,1


In [11]:
# Function to clean and split commodity string from the raw grouped data
def split_commodities(row):
    facility_type = row["facility_type"]
    entry = row["commodities"]
    count = row["count"]
    
    # Remove anything in parentheses
    entry = re.sub(r"\(.*?\)", "", entry)
    # Split on common delimiters (comma, slash, semicolon, " and ", etc.)
    split_items = re.split(r",|;|/|&| and ", entry)
    # Clean and filter items
    cleaned_items = [item.strip().lower() for item in split_items if item.strip()]
    
    return [(facility_type.lower(), item, count) for item in cleaned_items]

# Apply splitting and flatten the result
split_records = []
for _, row in raw_counts_grouped.iterrows():
    split_records.extend(split_commodities(row))

# Create DataFrame and group by to aggregate counts
split_df = pd.DataFrame(split_records, columns=["facility_type", "commodity", "count"])
grouped_split_df = split_df.groupby(["facility_type", "commodity"], as_index=False)["count"].sum()

In [12]:
grouped_split_df

,facility_type,commodity,count
0,manufacturing,alumina,2
1,manufacturing,aluminum,9
2,manufacturing,aluminum chemicals,1
3,manufacturing,ammonium sulfate,1
4,manufacturing,bismuth,1
...,...,...,...
99,project,uranium,7
100,project,vanadium,5
101,project,zinc,27
102,project,zircon,1


In [13]:
grouped_split_df.to_csv(r'results/data_coverage/commodities_covered.csv', index=False)

### Figure

In [39]:
from matplotlib import rcParams

In [23]:
commodities_covered = pd.read_csv(r'results/data_coverage/commodities_covered.csv')

In [50]:
commodities_covered

,facility_type,commodity,commodity_mapping,count
0,manufacturing,alumina,aluminum,2
1,manufacturing,aluminum,aluminum,9
2,manufacturing,aluminum chemicals,aluminum,1
3,manufacturing,ammonium sulfate,ammonium,1
4,manufacturing,bismuth,bismuth,1
...,...,...,...,...
99,project,uranium,uranium,7
100,project,vanadium,vanadium,5
101,project,zinc,zinc,27
102,project,zircon,zircon,1


In [49]:
commodities_covered['commodity_mapping'].nunique()

49

In [51]:
commodities_covered_active = commodities_covered[commodities_covered["facility_type"].str.strip() != "project"]

In [52]:
commodities_covered_active['commodity_mapping'].nunique()

34

In [45]:
# Apply the filtering to remove the dash entry from commodity_mapping
commodities_covered_clean = commodities_covered[commodities_covered["commodity_mapping"].str.strip() != "-"]

# Aggregate to recreate coverage_summary
coverage_summary = commodities_covered_clean.groupby(["facility_type", "commodity_mapping"])["count"].sum().reset_index()

# Sort for plotting
sorted_commodities = coverage_summary.groupby("commodity_mapping")["count"].sum().sort_values(ascending=False).index
coverage_summary["commodity_mapping"] = pd.Categorical(
    coverage_summary["commodity_mapping"], categories=sorted_commodities, ordered=True
)

# Define color palette
facility_palette = {
    "mining": "#393e41",
    "project": "#c8d5b9",
    "manufacturing": "#e94f37"
}

# Set matplotlib params
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans', 'Liberation Sans']
rcParams['font.size'] = 10
rcParams['axes.labelsize'] = 10
rcParams['legend.fontsize'] = 9
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['axes.titlesize'] = 12
sns.reset_orig()

# Plot
fig, ax = plt.subplots(figsize=(10, 4))

sns.barplot(
    data=coverage_summary,
    x="commodity_mapping",
    y="count",
    hue="facility_type",
    palette=facility_palette,
    ax=ax
)

ax.set_xlabel("")
ax.set_ylabel("Number of facilities")

# Add separators
for tick in range(len(sorted_commodities)):
    ax.axvline(x=tick - 0.5, color="lightgray", linestyle=":", linewidth=0.5)

ax.tick_params(axis='x', rotation=90)
ax.legend(title='', loc="upper right", frameon=False)
fig.tight_layout()

# Save
final_export_path = "results/data_coverage/commodities_covered.pdf"
fig.savefig(final_export_path, dpi=600, bbox_inches='tight', format='pdf')

# Heatmap of satellite data coverage by reporting level

In [ ]:
def heatmap(excel_path, base_save_path=None, dpi=600, max_cells_annotate=300):
    """
    Generate one heatmap per reporting level using data from extracted_data.xlsx.

    Parameters:
    - excel_path (str): Path to extracted_data.xlsx
    - base_save_path (str or None): Prefix for saved PNG files (e.g., "heatmap"). Appends level as suffix.
    - dpi (int): Image resolution
    - max_cells_annotate (int): Max annotated cells to prevent overcrowding
    """

    # Load all necessary sheets
    env_df = pd.read_excel(excel_path, sheet_name="Env")
    reserves_df = pd.read_excel(excel_path, sheet_name="Reserves")
    production_df = pd.read_excel(excel_path, sheet_name="Production")
    energy_df = pd.read_excel(excel_path, sheet_name="Energy")

    # ENVIRONMENTAL DATA
    env_df_clean = env_df.assign(
        type_group="environmental",
        type=env_df["type"].str.strip(),
        reporting_level=env_df["reporting_level"].str.strip(),
        commodities=env_df["commodity"].str.strip()
    )[["commodities", "type", "type_group", "reporting_level"]]

    # RESERVES DATA
    reserves_df_clean = reserves_df.assign(
        type_group="reserves",
        type=reserves_df["reserves_resources"].str.strip(),
        reporting_level=reserves_df["reporting_level"].str.strip(),
        commodities=reserves_df["commodity"].str.strip()
    )[["commodities", "type", "type_group", "reporting_level"]]

    # PRODUCTION DATA
    production_df_clean = production_df.assign(
        type_group="production",
        type=production_df["data_type"].str.strip(),
        reporting_level=production_df["level"].str.strip(),
        commodities=production_df["commodity"].str.strip()
    )[["commodities", "type", "type_group", "reporting_level"]]

    # ENERGY DATA
    energy_df_clean = energy_df.assign(
        type_group="energy",
        type="energy_flow",
        reporting_level=energy_df["reporting_level"].str.strip(),
        commodities=energy_df["commodity"].str.strip()
    )[["commodities", "type", "type_group", "reporting_level"]]

    # COMBINE ALL
    combined_df = pd.concat([
        env_df_clean,
        reserves_df_clean,
        production_df_clean,
        energy_df_clean
    ], ignore_index=True)

    # GROUP AND PLOT
    grouped = combined_df.groupby(['reporting_level', 'commodities', 'type']).size().reset_index(name='count')
    reporting_levels = grouped['reporting_level'].dropna().unique()

    for level in reporting_levels:
        data = grouped[grouped['reporting_level'] == level]
        pivot = data.pivot(index='commodities', columns='type', values='count').fillna(0)

        n_rows, n_cols = pivot.shape
        fig_width = min(0.5 * n_cols + 4, 40)
        fig_height = min(0.5 * n_rows + 4, 40)
        annotate = (n_rows * n_cols) <= max_cells_annotate

        plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
        sns.heatmap(
            pivot,
            annot=annotate,
            fmt=".0f",
            cmap="crest",
            linewidths=0.5,
            annot_kws={"size": 6} if annotate else None
        )
        plt.title(f"Data Coverage for Reporting Level: {level}", fontsize=14)
        plt.xlabel("", fontsize=12)
        plt.ylabel("Commodity", fontsize=12)
        plt.xticks(rotation=90, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()

        if base_save_path:
            safe_level = level.replace(" ", "_").replace("/", "_")
            save_path = f"{base_save_path}_{safe_level}.pdf"
            plt.savefig(save_path, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

In [ ]:
heatmap(
    excel_path="data/Tables/extracted_data.xlsx",
    base_save_path="results/data_coverage/heatmaps.pdf"
)
